In [41]:
import itertools
import numpy as np
import pandas as pd
import statsmodels.api as sm
from matplotlib import pyplot as plt
import seaborn as sns
import torch

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
# Render the figure in a notebook:
%matplotlib inline  

from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import plot_tree

from sklearn import metrics

from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torch.nn as nn
import torch.nn.functional as F

from torchmetrics.classification import Accuracy
from torchmetrics.classification import Recall
from torchmetrics import R2Score
from torchmetrics import MeanAbsoluteError

from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

In [77]:
x_train = pd.read_csv('../no_nontoxic_nans_X_train.csv',index_col=0)
y_train = pd.read_csv('../no_nontoxic_nans_y_train.csv',index_col=0)

x_val = pd.read_csv('../no_nontoxic_nans_x_val.csv')
y_val = pd.read_csv('../no_nontoxic_nans_y_val.csv',index_col=0)

x_test = pd.read_csv('../no_nontoxic_nans_x_test.csv')
y_test = pd.read_csv('../no_nontoxic_nans_y_test.csv',index_col=0)

### You have been using DataFrames because these are easy to deal with but here we need the data as an array
need to make a numpy array, followed by a tensor

In [79]:
#loads all the data into special pytorch data structures.

X_train_array = x_train.to_numpy()
X_train_tensor = torch.Tensor(X_train_array)

X_val_array = x_val.to_numpy()
X_val_tensor = torch.Tensor(X_val_array)

y_train_array = y_train.to_numpy()
y_train_tensor = torch.Tensor(y_train_array)

y_val_array = y_val.to_numpy()
y_val_tensor = torch.Tensor(y_val_array)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
trainloader = DataLoader(train_dataset, batch_size=128, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
valloader = DataLoader(val_dataset, batch_size=128, shuffle=True)

In [80]:
X_train_tensor

tensor([[ 0.7823,  0.7823, -0.4119,  ..., -0.0907, -0.2697, -0.1562],
        [ 0.5145,  0.5145, -0.7155,  ..., -0.0907, -0.2697, -0.1562],
        [-1.5689, -1.5689,  0.6186,  ..., -0.0907, -0.2697, -0.1562],
        ...,
        [ 0.2563,  0.2563, -0.5645,  ..., -0.0907, -0.2697, -0.1562],
        [ 0.2330,  0.2330, -0.1187,  ..., -0.0907, -0.2697, -0.1562],
        [ 0.7342,  0.7342, -0.7105,  ..., -0.0907, -0.2697, -0.1562]])

In [81]:
y_train_array

array([[0],
       [0],
       [1],
       ...,
       [0],
       [1],
       [1]], shape=(3691, 1))

In [97]:
y_train_tensor = y_train_tensor.squeeze()  # Converts shape (batch_size, 1) to (batch_size,)
y_val_tensor = y_val_tensor.squeeze()      # Converts shape (batch_size, 1) to (batch_size,)

# Ensure the target tensor is of type long (int64)
y_train_tensor = y_train_tensor.long()
y_val_tensor = y_val_tensor.long()
y_train_tensor

tensor([0, 0, 1,  ..., 0, 1, 1])

In [93]:
### effectively we are making something like LinearRegression ourselves
class MyPerceptron(nn.Module):    # this class inherits from nn.Module
    def __init__(self):
        super(MyPerceptron, self).__init__() #this calls the constructor of the parent class nn.Module
        
        # define network layers
        self.fc1 = nn.Linear(217, 1)   # nn.Linear is a class for linear layers, i.e. input to the first hidden layer
        self.fc2 = nn.Linear(1, 2) # this is the hidden layer to the output layer with two outputs (0 or 1)
        
        # Here we are using a linear model as the activation function
    def forward(self,x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [121]:
### set a random seed ###
torch.manual_seed(0)

### create the network set the criretia and how to optimize
net1 = MyPerceptron()
net1.train() ### this turns the model on for training (above there are no specific layers only involved in training but this is good practise)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net1.parameters(), lr=0.0001)

### set the number of epochs to run
num_epochs = 100  

for epoch in range(1,num_epochs+1):
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        inputs, labels = data ### extacts the features and ground truth labels
        optimizer.zero_grad() ### removes gradients from previous run in backpropergation
        outputs = net1(inputs) ### gets the predictions - which is now the raw scores "logits"
        labels = labels.squeeze().long()
        if inputs.size(0) < 128:
            continue  
        # print(f"Batch {i+1}: Inputs shape = {inputs.shape}, Labels shape = {labels.shape}")
        loss = criterion(outputs, labels.long()) ### calculates the loss of the outputs with the labels
        loss.backward() ### computes the gradients for backpropagation
        optimizer.step() ### updates the model based on the gradients
        running_loss += loss.item() ### adds the loss to the running_loss 

    avg_train_loss = running_loss / len(trainloader) ### calculates the average lss for the batches

    print('Epoch',epoch,'Loss =',avg_train_loss) ### prints the info
    

Epoch 1 Loss = nan
Epoch 2 Loss = nan
Epoch 3 Loss = nan
Epoch 4 Loss = nan
Epoch 5 Loss = nan
Epoch 6 Loss = nan
Epoch 7 Loss = nan
Epoch 8 Loss = nan
Epoch 9 Loss = nan
Epoch 10 Loss = nan
Epoch 11 Loss = nan
Epoch 12 Loss = nan
Epoch 13 Loss = nan
Epoch 14 Loss = nan
Epoch 15 Loss = nan
Epoch 16 Loss = nan
Epoch 17 Loss = nan
Epoch 18 Loss = nan
Epoch 19 Loss = nan
Epoch 20 Loss = nan
Epoch 21 Loss = nan
Epoch 22 Loss = nan
Epoch 23 Loss = nan
Epoch 24 Loss = nan
Epoch 25 Loss = nan
Epoch 26 Loss = nan
Epoch 27 Loss = nan
Epoch 28 Loss = nan
Epoch 29 Loss = nan
Epoch 30 Loss = nan
Epoch 31 Loss = nan
Epoch 32 Loss = nan
Epoch 33 Loss = nan
Epoch 34 Loss = nan
Epoch 35 Loss = nan
Epoch 36 Loss = nan
Epoch 37 Loss = nan
Epoch 38 Loss = nan
Epoch 39 Loss = nan
Epoch 40 Loss = nan
Epoch 41 Loss = nan
Epoch 42 Loss = nan
Epoch 43 Loss = nan
Epoch 44 Loss = nan
Epoch 45 Loss = nan
Epoch 46 Loss = nan
Epoch 47 Loss = nan
Epoch 48 Loss = nan
Epoch 49 Loss = nan
Epoch 50 Loss = nan
Epoch 51 

In [122]:
# Training loop
for epoch in range(1, num_epochs + 1):
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        inputs, labels = data
        
        # Skip small batches (e.g., if batch size is less than 128)
        if inputs.size(0) < 128:
            print(f"Skipping batch {i+1} due to small size ({inputs.size(0)} samples)")
            continue
        
        labels = labels.squeeze().long()

        optimizer.zero_grad()

        # Forward pass
        outputs = net1(inputs)

        # Check for NaN values in outputs
        if torch.any(torch.isnan(outputs)):
            print(f"NaN detected in outputs at batch {i+1}")
        
        # Check for NaN values in labels
        if torch.any(torch.isnan(labels)):
            print(f"NaN detected in labels at batch {i+1}")
        
        # Calculate loss
        loss = criterion(outputs, labels)

        # Check if the loss is NaN
        if torch.isnan(loss):
            print(f"NaN detected in loss at batch {i+1}")
            break  # Stop if loss becomes NaN
        
        loss.backward()

        # Gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(net1.parameters(), max_norm=1.0)
        
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(trainloader)
    print(f"Epoch {epoch}, Loss = {avg_train_loss:.4f}")

NaN detected in outputs at batch 1
NaN detected in loss at batch 1
Epoch 1, Loss = 0.0000
NaN detected in outputs at batch 1
NaN detected in loss at batch 1
Epoch 2, Loss = 0.0000
NaN detected in outputs at batch 1
NaN detected in loss at batch 1
Epoch 3, Loss = 0.0000
NaN detected in outputs at batch 1
NaN detected in loss at batch 1
Epoch 4, Loss = 0.0000
NaN detected in outputs at batch 1
NaN detected in loss at batch 1
Epoch 5, Loss = 0.0000
NaN detected in outputs at batch 1
NaN detected in loss at batch 1
Epoch 6, Loss = 0.0000
NaN detected in outputs at batch 1
NaN detected in loss at batch 1
Epoch 7, Loss = 0.0000
NaN detected in outputs at batch 1
NaN detected in loss at batch 1
Epoch 8, Loss = 0.0000
NaN detected in outputs at batch 1
NaN detected in loss at batch 1
Epoch 9, Loss = 0.0000
NaN detected in outputs at batch 1
NaN detected in loss at batch 1
Epoch 10, Loss = 0.0000
NaN detected in outputs at batch 1
NaN detected in loss at batch 1
Epoch 11, Loss = 0.0000
NaN dete

In [106]:
y_train_tensor.shape

torch.Size([3691])

In [107]:
y_train_tensor.dtype

torch.int64

In [105]:
outputs = net1(inputs)
outputs.shape

torch.Size([128, 2])

In [114]:
labels.shape

torch.Size([107])